# Evaluate Results

## Correct Weighted f1

In [1]:
from decision_trees_run import ClassifierDecisionTree
from decision_trees_run import NeuronPredictorArgs
from decision_trees_run import NeuronPredictor
from decision_trees_run import NeuronPredictorEvaluator

from tqdm import tqdm
import numpy as np
import pandas as pd
import os

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [2]:
layer = 0
df = pd.read_csv(f"results_decision_tree_classifier_L{layer}_correct_weighted_f1.csv")
df[(df["weighted_f1"] > 1.0) | (df["weighted_f1_train"] > 1.0)]

,Unnamed: 0,layer,neuron,min_impurity_decrease,f1,f1_train,weighted_f1,weighted_f1_train,variable_count,rules


## Evaluate Results

### Create Table
- Remove expamples with 0.0000 F1 (Not really remove but don't count in the score ...)
- TODO: Investigate what exactly happens, when f1 is 0.0: I think the DT does not find a single split with the correct impurity decrease, then the dt predicts NONE ...
- Layer, min_impurity_decrease, Avg F1, Avg F1_train, Avg weighted_F1, Avg weighted_F1_train, Avg Variable Count, Fraction of 0.0000

In [3]:
# Remove expamples with 0.0000 F1 (Not really remove but don't count in the score ...)
# TODO: Investigate what exactly happens, when f1 is 0.0: I think the DT does not find a single split with the correct impurity decrease, then the dt predicts NONE ...
# Layer, min_impurity_decrease, Avg F1, Avg F1_train, Avg weighted_F1, Avg weighted_F1_train, Avg Variable Count, Fraction of 0.0000

min_impurity_decreases = [0.0001, 0.0005]
summary_dict = {
    "layer": [],
    "min_impurity_decrease": [],
    "f1": [],
    "f1_train": [],
    "weighted_f1": [],
    "weighted_f1_train": [],
    "variable_count": [],
    "fraction_0": []
}

for layer in range(8):
    df = pd.read_csv(f"results_decision_tree_classifier_L{layer}_correct_weighted_f1.csv")
    for min_impurity_decrease in min_impurity_decreases:
        df_small = df[df["min_impurity_decrease"] == min_impurity_decrease]
        df_real = df_small[df_small["f1"] != 0]
        f1 = df_real["f1"].mean()
        f1_train = df_real["f1_train"].mean()
        weighted_f1 = df_real["weighted_f1"].mean()
        weighted_f1_train = df_real["weighted_f1_train"].mean()
        variable_count = df_small["variable_count"].mean()
        fraction_0 = len(df_small[df_small["f1"] == 0]) / len(df_small)
        summary_dict["layer"].append(layer)
        summary_dict["min_impurity_decrease"].append(min_impurity_decrease)
        summary_dict["f1"].append(f1)
        summary_dict["f1_train"].append(f1_train)
        summary_dict["weighted_f1"].append(weighted_f1)
        summary_dict["weighted_f1_train"].append(weighted_f1_train)
        summary_dict["variable_count"].append(variable_count)
        summary_dict["fraction_0"].append(fraction_0)
        print(f"Layer {layer}, min_impurity_decrease {min_impurity_decrease}, Avg F1 {f1}, Avg F1_train {f1_train}, Avg weighted_F1 {weighted_f1}, Avg weighted_F1_train {weighted_f1_train}, Avg Variable Count {variable_count}, Fraction of 0.0000 {fraction_0}")
        # save df with new path

summary_df = pd.DataFrame(summary_dict)
summary_df.to_csv(f"results__summary_decision_tree_classifier_L{layer}.csv")
summary_df.round(2)

Layer 0, min_impurity_decrease 0.0001, Avg F1 0.730228663026378, Avg F1_train 0.7478350260945655, Avg weighted_F1 0.8287108610645415, Avg weighted_F1_train 0.7478345397455269, Avg Variable Count 93.16796875, Fraction of 0.0000 0.01171875
Layer 0, min_impurity_decrease 0.0005, Avg F1 0.6638265699814664, Avg F1_train 0.6683719555900627, Avg weighted_F1 0.7643310597321415, Avg weighted_F1_train 0.6683714881060043, Avg Variable Count 26.93359375, Fraction of 0.0000 0.09423828125
Layer 1, min_impurity_decrease 0.0001, Avg F1 0.6572634065325177, Avg F1_train 0.6751180189646707, Avg weighted_F1 0.7686170890815224, Avg weighted_F1_train 0.6751175390755331, Avg Variable Count 147.7421875, Fraction of 0.0000 0.0107421875
Layer 1, min_impurity_decrease 0.0005, Avg F1 0.5880705533659405, Avg F1_train 0.5920865856074792, Avg weighted_F1 0.692872264635579, Avg weighted_F1_train 0.5920861290271986, Avg Variable Count 34.42578125, Fraction of 0.0000 0.10009765625
Layer 2, min_impurity_decrease 0.0001,

,layer,min_impurity_decrease,f1,f1_train,weighted_f1,weighted_f1_train,variable_count,fraction_0
0,0,0.0,0.73,0.75,0.83,0.75,93.17,0.01
1,0,0.0,0.66,0.67,0.76,0.67,26.93,0.09
2,1,0.0,0.66,0.68,0.77,0.68,147.74,0.01
3,1,0.0,0.59,0.59,0.69,0.59,34.43,0.10
4,2,0.0,0.66,0.68,0.76,0.68,139.47,0.02
5,2,0.0,0.61,0.61,0.70,0.61,32.64,0.15
6,3,0.0,0.67,0.68,0.76,0.68,120.73,0.04
7,3,0.0,0.65,0.65,0.73,0.65,28.72,0.21
8,4,0.0,0.73,0.74,0.81,0.74,96.76,0.04
9,4,0.0,0.73,0.73,0.81,0.73,26.09,0.19


- Layer 6 has very High rates of f1 0.0. This is surprising, because I thought they would be very interpretabil ...
- F1 is 0.0 if no rule is found with probability > 0.5 (making min_impurity_decrease smaller can make this go away but f1 score is still bad so ...)

In [8]:
# round all columns except min_impurity_decrease
summary_df.loc[:, ["f1", "f1_train", "weighted_f1", "fraction_0"]] = summary_df.loc[:, ["f1", "f1_train", "weighted_f1", "fraction_0"]].round(2)
summary_df.loc[:, ["variable_count"]] = summary_df.loc[:, ["variable_count"]].round(0)

,layer,min_impurity_decrease,f1,f1_train,weighted_f1,weighted_f1_train,variable_count,fraction_0
0,0,0.0001,0.73,0.75,0.83,0.747835,93.0,0.01
1,0,0.0005,0.66,0.67,0.76,0.668371,27.0,0.09
2,1,0.0001,0.66,0.68,0.77,0.675118,148.0,0.01
3,1,0.0005,0.59,0.59,0.69,0.592086,34.0,0.10
4,2,0.0001,0.66,0.68,0.76,0.675365,139.0,0.02
5,2,0.0005,0.61,0.61,0.70,0.613046,33.0,0.15
6,3,0.0001,0.67,0.68,0.76,0.681330,121.0,0.04
7,3,0.0005,0.65,0.65,0.73,0.651085,29.0,0.21
8,4,0.0001,0.73,0.74,0.81,0.737787,97.0,0.04
9,4,0.0005,0.73,0.73,0.81,0.731484,26.0,0.19


In [11]:
summary_df.drop(columns=["weighted_f1_train"], inplace=True)

In [12]:
summary_df

,layer,min_impurity_decrease,f1,f1_train,weighted_f1,variable_count,fraction_0
0,0,0.0001,0.73,0.75,0.83,93.0,0.01
1,0,0.0005,0.66,0.67,0.76,27.0,0.09
2,1,0.0001,0.66,0.68,0.77,148.0,0.01
3,1,0.0005,0.59,0.59,0.69,34.0,0.10
4,2,0.0001,0.66,0.68,0.76,139.0,0.02
5,2,0.0005,0.61,0.61,0.70,33.0,0.15
6,3,0.0001,0.67,0.68,0.76,121.0,0.04
7,3,0.0005,0.65,0.65,0.73,29.0,0.21
8,4,0.0001,0.73,0.74,0.81,97.0,0.04
9,4,0.0005,0.73,0.73,0.81,26.0,0.19


In [13]:
print(summary_df.to_latex(index=False))

\begin{tabular}{rrrrrrr}
\toprule
layer & min_impurity_decrease & f1 & f1_train & weighted_f1 & variable_count & fraction_0 \\
\midrule
0 & 0.000100 & 0.730000 & 0.750000 & 0.830000 & 93.000000 & 0.010000 \\
0 & 0.000500 & 0.660000 & 0.670000 & 0.760000 & 27.000000 & 0.090000 \\
1 & 0.000100 & 0.660000 & 0.680000 & 0.770000 & 148.000000 & 0.010000 \\
1 & 0.000500 & 0.590000 & 0.590000 & 0.690000 & 34.000000 & 0.100000 \\
2 & 0.000100 & 0.660000 & 0.680000 & 0.760000 & 139.000000 & 0.020000 \\
2 & 0.000500 & 0.610000 & 0.610000 & 0.700000 & 33.000000 & 0.150000 \\
3 & 0.000100 & 0.670000 & 0.680000 & 0.760000 & 121.000000 & 0.040000 \\
3 & 0.000500 & 0.650000 & 0.650000 & 0.730000 & 29.000000 & 0.210000 \\
4 & 0.000100 & 0.730000 & 0.740000 & 0.810000 & 97.000000 & 0.040000 \\
4 & 0.000500 & 0.730000 & 0.730000 & 0.810000 & 26.000000 & 0.190000 \\
5 & 0.000100 & 0.760000 & 0.770000 & 0.850000 & 98.000000 & 0.050000 \\
5 & 0.000500 & 0.760000 & 0.770000 & 0.860000 & 28.000000 & 0.150000 

In [ ]:
# Next we look at different correlations.
# We want to show that the Neuron Activation Distribution / Summary Statistics tell us which neurons follow rules and which don't
# This is evidence that our decision tree actually finds all the neurons that are understandable and this is as good as it gets.

In [ ]:
# use plotly to plot the correlation matrix

In [ ]:
# use plotly to plot a scatter plot with skewness and weighted f1 score as x and y axis

In [14]:
import pandas as pd
from tqdm import tqdm
import torch as t
import plotly.express as px
import plotly.graph_objects as go
# load datasets
big_datasets = []
for layer in tqdm(range(8)):
    df = pd.read_csv(f"data/neuron_datasets/big_argmax_train_L{layer}.csv")
    big_datasets.append(df)
# load datasets
big_datasets_test = []
for layer in tqdm(range(8)):
    df = pd.read_csv(f"data/neuron_datasets/big_argmax_test_L{layer}.csv")
    big_datasets_test.append(df)

100%|██████████| 8/8 [02:18<00:00, 17.33s/it]


In [15]:
average = t.zeros((8, 2048))
std = t.zeros((8, 2048))
skewness = t.zeros((8, 2048))
kurtosis = t.zeros((8, 2048))
fraction = t.zeros((8, 2048))
for layer in tqdm(range(8)):
    df = big_datasets[layer]
    for neuron in range(2048):
        neuron_activation = df[f"L{layer}_N{neuron}"]
        neuron_activation_positive = neuron_activation[neuron_activation > 0]
        average[layer, neuron] = neuron_activation_positive.mean()
        std[layer, neuron] = neuron_activation_positive.std()
        skewness[layer, neuron] = neuron_activation_positive.skew()
        kurtosis[layer, neuron] = neuron_activation_positive.kurtosis()
        fraction[layer, neuron] = len(neuron_activation_positive) / len(neuron_activation)

100%|██████████| 8/8 [00:16<00:00,  2.01s/it]


In [43]:
f1_scores_0001 = t.zeros((8, 2048))
variable_count_0001 = t.zeros((8, 2048))
f1_scores_0005 = t.zeros((8, 2048))
variable_count_0005 = t.zeros((8, 2048))
for layer in tqdm(range(8)):
    df = pd.read_csv(f"results_decision_tree_classifier_L{layer}_correct_weighted_f1.csv")
    for neuron in range(2048):
        f1_scores_0001[layer, neuron] = df[(df["neuron"] == neuron) & (df["min_impurity_decrease"] == 0.0001)]["weighted_f1"].mean()
        f1_scores_0005[layer, neuron] = df[(df["neuron"] == neuron) & (df["min_impurity_decrease"] == 0.0005)]["weighted_f1"].mean()
        variable_count_0001[layer, neuron] = df[(df["neuron"] == neuron) & (df["min_impurity_decrease"] == 0.0001)]["variable_count"].mean()
        variable_count_0005[layer, neuron] = df[(df["neuron"] == neuron) & (df["min_impurity_decrease"] == 0.0005)]["variable_count"].mean()

100%|██████████| 8/8 [00:23<00:00,  2.89s/it]


In [48]:
# create the correlation matrix of average, std, skewness, kurtosis, fraction, f1_scores_0001, f1_scores_0005, variable_count_0001, variable_count_0005
average_flat = average.flatten()
std_flat = std.flatten()
skewness_flat = skewness.flatten()
kurtosis_flat = kurtosis.flatten()
fraction_flat = fraction.flatten()
f1_scores_0001_flat = f1_scores_0001.flatten()
f1_scores_0005_flat = f1_scores_0005.flatten()
variable_count_0001_flat = variable_count_0001.flatten()
variable_count_0005_flat = variable_count_0005.flatten()

correlation_df = pd.DataFrame({
    "average": average_flat,
    "std": std_flat,
    "skewness": skewness_flat,
    "kurtosis": kurtosis_flat,
    "fraction": fraction_flat,
    "f1_scores_0001": f1_scores_0001_flat,
    "f1_scores_0005": f1_scores_0005_flat,
    "variable_count_0001": variable_count_0001_flat,
    "variable_count_0005": variable_count_0005_flat
})

correlation_df.corr()
# plot the correlation matrix with plotly
fig = px.imshow(correlation_df.corr())
fig.show()
# negative correlation beween skewness and f1! (nice)
# positive correlation between variable count and f1
# negative correlation between std and variable count
# print these correlations
print(f"Correlation between skewness and f1: {correlation_df['skewness'].corr(correlation_df['f1_scores_0001'])}")
print(f"Correlation between variable count and f1: {correlation_df['variable_count_0001'].corr(correlation_df['f1_scores_0001'])}")
print(f"Correlation between std and variable count: {correlation_df['std'].corr(correlation_df['variable_count_0001'])}")

Correlation between skewness and f1: -0.46054090053595875
Correlation between variable count and f1: 0.2522676539122705
Correlation between std and variable count: -0.47987763803768846


In [41]:
real_layer = 2
fig = go.Figure()
for layer in range(real_layer, real_layer+1):
    fig.add_trace(go.Scatter(x=f1_scores_0005[layer], y=skewness[layer], mode="markers", name=f"Layer {layer}", text=[f"Neuron {i}" for i in range(2048)]))
    fig.update_layout(
    )
fig.show()

In [ ]:
# use plotly to plot the correlation matrix of the different statistics


In [40]:
df_layer_6 = pd.read_csv("results_decision_tree_classifier_L6_correct_weighted_f1.csv")
df_layer_6[(df_layer_6["f1"] == 0) & (df_layer_6["min_impurity_decrease"] == 0.0001)]

,Unnamed: 0,layer,neuron,min_impurity_decrease,f1,f1_train,weighted_f1,weighted_f1_train,variable_count,rules
18,18,6,9,0.0001,0.0,0.0,0.0,0.0,1,['=> 0.0022']
22,22,6,11,0.0001,0.0,0.0,0.0,0.0,5,"['F3 not accesible G5 not empty => 0.0478', 'F..."
28,28,6,14,0.0001,0.0,0.0,0.0,0.0,1,['=> 0.0003']
32,32,6,16,0.0001,0.0,0.0,0.0,0.0,1,['=> 1.0000']
34,34,6,17,0.0001,0.0,0.0,0.0,0.0,1,['=> 0.0008']
...,...,...,...,...,...,...,...,...,...,...
4066,4066,6,2033,0.0001,0.0,0.0,0.0,0.0,9,"['F4 mine H5 not empty G5 yours => 0.0872', 'F..."
4074,4074,6,2037,0.0001,0.0,0.0,0.0,0.0,1,['=> 0.0000']
4080,4080,6,2040,0.0001,0.0,0.0,0.0,0.0,1,['=> 0.0018']
4086,4086,6,2043,0.0001,0.0,0.0,0.0,0.0,1,['=> 0.0001']


In [5]:
layer = 6
bis_dataset_train = pd.read_csv(f"data/neuron_datasets/big_argmax_train_L{layer}.csv")
big_dataset_test = pd.read_csv(f"data/neuron_datasets/big_argmax_test_L{layer}.csv")

In [72]:
neuron = 11

args = NeuronPredictorArgs(
    dataset_kind = "big_argmax",
    neuron_predictor_type = "decision_tree_classifier",
    preprocess_train_type = "round",
    preprocess_eval_type = "normal",
    layer = layer,
    neuron = neuron,
    len_data = 100000,
    dataset_train = bis_dataset_train,
    dataset_test = big_dataset_test,
)
evaluator_round = NeuronPredictorEvaluator(args)
weighted_f1, weighted_f1_train, f1, f1_train, variable_count, rules = evaluator_round.train_and_evaluate(min_impurity_decrease=0.00002)

In [73]:
f1, weighted_f1, f1_train, weighted_f1_train, variable_count

(0.07004310344827586,
 0.08131008190493504,
 0.16933402168301498,
 0.16933385385343597,
 269)

In [68]:
rules

['F3 not empty AND G5 not empty => 0.0478',
 'F3 not empty AND G5 empty => 0.0296',
 'F3 empty => 0.0125']

In [59]:
import plotly.express as px
layer = 6
neuron = 9
print(layer)

activations = big_dataset_test[f"L{layer}_N{neuron}"]
activations = activations[activations > 0]
print(len(activations))
fig = px.histogram(x=activations)
fig.update_layout(
    title=f"Activations of Neurons in Layer {layer} of the MLP",
    xaxis_title="Activations",
    yaxis_title="Count",
)
fig.show()

6
126


In [21]:
rules.

['F0 placed AND F4 legal AND E5 legal AND A6 not empty => 1.0000',
 'H2 placed AND F4 legal AND E5 legal AND D0 legal => 1.0000',
 'H2 not placed AND F0 not placed => 0.0000',
 'F0 placed AND F4 not legal => 0.0000',
 'F0 placed AND F4 legal AND E5 not legal => 0.0000',
 'F0 placed AND F4 legal AND E5 legal AND A6 empty => 0.0000',
 'H2 placed AND F4 not legal => 0.0000',
 'H2 placed AND F4 legal AND E5 not legal => 0.0000',
 'H2 placed AND F4 legal AND E5 legal AND D0 not legal => 0.0000']

In [22]:
f1

0.0